In [42]:
import ee
service_account = 'clgx-gis-app-dev-06e3@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'clgx-gis-app-dev-06e3-a1a85a16331a.json')
ee.Initialize(credentials)

In [43]:
import geopandas as gpd
from osgeo import gdal
import geemap
import glob
import os
import numpy as np

URl google storage cat1 clips to 12071
gs://geospatial-team/abreunig/Projects/gee_poc

In [44]:
small_bbox = gpd.read_file('stormsurge_data/12071_small_bbox.shp')
xmin, ymin, xmax, ymax = small_bbox.total_bounds
region = ee.Geometry.BBox(xmin, ymin, xmax, ymax)

In [11]:
bbox = gpd.read_file('stormsurge_data/12071.shp')
xmin, ymin, xmax, ymax = bbox.total_bounds
print(xmin, ymin, xmax, ymax)

-82.28276899963568 26.316180000289584 -81.56216000000444 26.79001399960236


In [21]:
output_dir = r'D:\GitHub-private_Repos\gee-python\stormsurge_data'
for fi in glob.glob('*/*/*.tif'):
    input_rast = gdal.Open(fi)
    name = os.path.basename(fi).split('_')[1]
    aoi_rast = gdal.Translate(destName=os.path.join(output_dir, f'{name}_12071.tif'), 
                                srcDS=input_rast, 
                                projWin=[xmin, ymax, xmax, ymin],
                                outputSRS='EPSG:4326',
                                creationOptions=['COMPRESS=LZW', 'TILED=YES', 'COPY_SRC_OVERVIEWS=YES'])

    
#-co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW

In [85]:
cat1 = ee.Image.loadGeoTIFF('gs://geospatial-team/abreunig/Projects/gee_poc/Category1_12071.tif')
cat2 = ee.Image.loadGeoTIFF('gs://geospatial-team/abreunig/Projects/gee_poc/Category2_12071.tif')
cat3 = ee.Image.loadGeoTIFF('gs://geospatial-team/abreunig/Projects/gee_poc/Category3_12071.tif')
cat4 = ee.Image.loadGeoTIFF('gs://geospatial-team/abreunig/Projects/gee_poc/Category4_12071.tif')
cat5 = ee.Image.loadGeoTIFF('gs://geospatial-team/abreunig/Projects/gee_poc/Category5_12071.tif')

In [86]:
# cahnge pixeltype to int16
cat1 = cat1.toFloat()
cat2 = cat2.toFloat()

In [87]:
import io
import requests

In [91]:
url = cat1.getDownloadUrl({
    #'bands': ['B0'],
    'region': region,
    'format': 'NPY'
})
response = requests.get(url)
data = np.load(io.BytesIO(response.content))
# convert data to int64
data = data.astype(np.float32)

In [92]:
data[0]

array([4., 4., 3., 2., 2., 3., 4., 4., 5., 5., 5., 5., 5., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 7., 7., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [90]:
# convert 255 values to 1 for cat1 and 2 for cat2
def convert255(image, value):
    return image.where(image.eq(255), value)
#cat1 = cat1.toArray()
#cat2 = cat2.toArray()

cat1 = convert255(cat1, 10)
cat2 = convert255(cat2, 10)

In [83]:
# add values of cat1 and cat2
first_inundation = cat1.add(cat2)


In [84]:
# change values in first_inundation to 2 if value not negative
first_inundation = first_inundation.where(first_inundation.gte(0), 2)

In [51]:

CAT1_DEP = 10 # first slot after decimal
CAT2_DEP = 100 # second slot after decimal
CAT3_DEP = 1000 # third slot after decimal
CAT4_DEP = 10000 # fourth slot after decimal
CAT5_DEP = 100000 # fifth slot after decimal

cat12 = (
    ee.Image(first_inundation)
    .add(cat1.divide(CAT1_DEP))
    .add(cat2.divide(CAT2_DEP))
)

In [52]:
cat12.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B0',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -270.0800476074219,
    'max': 267.97003173828125},
   'dimensions': [8022, 5275],
   'crs': 'GEOGCS["WGS 84", \n  DATUM["World Geodetic System 1984", \n    SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n    AUTHORITY["EPSG","6326"]], \n  PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Geodetic longitude", EAST], \n  AXIS["Geodetic latitude", NORTH], \n  AUTHORITY["EPSG","4326"]]',
   'crs_transform': [8.983152799999972e-05,
    0,
    -82.2828439716901,
    0,
    -8.983152799999973e-05,
    26.790014051375877]}]}

In [81]:
import math

In [76]:
first_inudation = 2 # cat1 inudation
cat1_depth = 0
cat2_depth = 3
cat3_depth = 5
cat4_depth = 7
cat5_depth = 9


CAT1_DEP = 10 # first slot after decimal
CAT2_DEP = 100 # second slot after decimal
CAT3_DEP = 1000 # third slot after decimal
CAT4_DEP = 10000 # fourth slot after decimal
CAT5_DEP = 100000 # fifth slot after decimal

# create staked value where: cat . cat1_depth cat2_depth cat3_depth cat4_depth cat5_depth
value = (first_inudation + (
    cat1_depth / CAT1_DEP + 
    cat2_depth / CAT2_DEP + 
    cat3_depth / CAT3_DEP + 
    cat4_depth / CAT4_DEP + 
    cat5_depth / CAT5_DEP)
)

print(value)

2.03579


In [54]:

# Specify the GCS bucket and folder where you want to save the image
bucket = 'geospatial-team'
folder = 'abreunig/Projects/gee_poc'

# Specify the name for the exported image
image_name = 'cat12'

# Create a GCS export configuration object
export_config = {
    'image': cat12,
    'description': 'Image Export',
    'bucket': bucket,
    'fileNamePrefix': folder + '/' + image_name,
    'region': region,
    'scale': 10  # Adjust the scale as needed
}

# Start the export
task = ee.batch.Export.image.toCloudStorage(**export_config)
task.start()

# Monitor the task status
while task.status()['state'] in ['READY', 'RUNNING']:
    print('Exporting...')

print('Export completed:', task.status())


Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...

In [35]:
# A function to print arrays for a selected pixel in the following examples.
def samp_arr_img(arr_img):
  point = ee.Geometry.Point([-121, 42])
  return arr_img.sample(point, 500).first().get('array')


# A 3-band image of constants.
img = ee.Image([0, 1, 2])
display('3-band image', img)

# Convert the 3-band image to an array image. The resulting array image has a
# single band named "array". The "array" band stores the per-pixel band values
# from the input ee.Image as a 1D array.
array_img_1_d = img.toArray()
display('1D array image', array_img_1_d)

# Sample a single pixel to see its 1D array using the `samp_arr_img` function
# defined above. Similar arrays are present for all pixels in a given array
# image looking at just one helps conceptualize the structure.
display('1D array image (pixel)', samp_arr_img(array_img_1_d))
# [0, 1, 2]

# Array images can be displayed to the Code Editor map and queried with the
# inspector tool. Per-pixel, the first element in the array is shown.
# Single-band grayscale is used to represent the data. Style parameters `min`
# and `max` are valid. Styling the layer with the Code Editor's layer
# visualization tool is invalid.
m = geemap.Map()
m.add_layer(array_img_1_d, {'min': 0, 'max': 2}, 'Image array')
display(m)

# Create a 2D array image by concatenating the values in a 1D array image
# along the 1-axis using `toArray(1)`. For a 3D array, apply `toArray(2)` to
# the result.
array_img_2_d = array_img_1_d.toArray(1)
display('2D array image (pixel)', samp_arr_img(array_img_2_d))
# [[0],
#  [1],
#  [2]]

'3-band image'

'1D array image'

'1D array image (pixel)'

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

'2D array image (pixel)'